In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os

In [3]:
path = r'C:\Users\nickr\OneDrive\Documents\Nick Bootcamp\Python\Instacart Basket Analysis'

In [5]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined_new_variables.pkl'))

In [7]:
df_ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.562844
2,17.518895
3,16.907152
4,17.537934
5,14.894521
6,16.658449
7,17.072954
8,14.942021


The mean did not differ from the subset.

# Creating Loyalty Customer Flag

In [11]:
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\nickr\AppData\Local\Temp\ipykernel_13552\136815596.py:1: FutureWarning: The provided callable <function max at 0x0000016EB4F920C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [13]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [15]:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    473041
Loyal customer      288151
New customer        238808
Name: count, dtype: int64

# Loyalty flags' mean, max, min

In [40]:
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.781255  1.0  24.5
New customer      7.889919  1.0  24.5
Regular customer  7.859615  1.0  24.5

Grouping the data by loyalty flag and aggregating it by price to include mean, min, and max gave the above result.
Loyal customers have a slightly lower mean than new and regular customers. However, they are about the same with no noticeable difference.

# Creating a Type of Spender Flag

In [43]:
df_ords_prods_merge['type_of_spender'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [45]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_days_of_week,hour_of_day_ordered,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range,Busiest_Days,Busiest_period_of_day,max_order,loyalty_flag,type_of_spender
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Most Orders,30,Regular customer,12.900000
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Most Orders,30,Regular customer,12.900000
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Average Orders,5,New customer,5.288889
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,both,Mid-range product,Slowest day,Most Orders,3,New customer,5.800000
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Slowest day,Most Orders,3,New customer,5.800000


# Creating Type of Spender Name to assign to each user id

In [52]:
df_ords_prods_merge.loc[df_ords_prods_merge['type_of_spender'] < 10, 'spender_flag'] = 'Low Spender'
df_ords_prods_merge.loc[df_ords_prods_merge['type_of_spender'] >= 10, 'spender_flag'] = 'High Spender'

In [58]:
df_ords_prods_merge['spender_flag'].value_counts(dropna = False)

spender_flag
Low Spender     791539
High Spender    208461
Name: count, dtype: int64

# Creating an Order Frequency Flag

In [66]:
df_ords_prods_merge['order_frequency'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [68]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_days_of_week,hour_of_day_ordered,...,reordered,_merge,price_range,Busiest_Days,Busiest_period_of_day,max_order,loyalty_flag,type_of_spender,spender_flag,order_frequency
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,0,both,Mid-range product,Regularly busy,Most Orders,30,Regular customer,12.900000,High Spender,6.5
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,both,Mid-range product,Regularly busy,Most Orders,30,Regular customer,12.900000,High Spender,6.5
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,0,both,Mid-range product,Busiest day,Average Orders,5,New customer,5.288889,Low Spender,7.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,0,both,Mid-range product,Slowest day,Most Orders,3,New customer,5.800000,Low Spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,1,both,Mid-range product,Slowest day,Most Orders,3,New customer,5.800000,Low Spender,9.0


# Creating Frequent Customer tag to assign to each user id

In [70]:
df_ords_prods_merge.loc[df_ords_prods_merge['order_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['order_frequency'] <= 20) & (df_ords_prods_merge['order_frequency'] > 10), 'frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [72]:
df_ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        641879
Regular customer         220918
Non-frequent customer    129006
NaN                        8197
Name: count, dtype: int64

In [74]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_ords_prods_merge_group_agg.pkl'))